In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import jieba
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

path = './text_similarity/'
# 数据加载
news = pd.read_csv(path+'sqlResult.csv',encoding='gb18030')
# 处理缺失值
print(news[news.content.isna()].head(5))
news=news.dropna(subset=['content'])

# 加载停用词
with open(path+'chinese_stopwords.txt','r', encoding='utf-8') as file:
    stopwords=[i[:-1] for i in file.readlines()]
    
# 分词
def split_text(text):
    #return ' '.join([w for w in list(jieba.cut(re.sub('\s|[%s]' % (punctuation),'',text))) if w not in stopwords])
    text = text.replace(' ', '')
    text = text.replace('\n', '')
    text2 = jieba.cut(text.strip())
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

# 加载清洗后的corpus
if not os.path.exists(path+"corpus.pkl"):
    # 对所有文本进行分词
    corpus=list(map(split_text,[str(i) for i in news.content]))
    print(corpus[0])
    print(len(corpus))
    print(corpus[1])
    # 保存到文件，方便下次调用
    with open(path+'corpus.pkl','wb') as file:
        pickle.dump(corpus, file)
else:
    # 调用上次处理的结果
    with open(path+'corpus.pkl','rb') as file:
        corpus = pickle.load(file)

# 得到corpus的TF-IDF矩阵
if not os.path.exists(path+"tfidf.pkl"):
    countvectorizer = CountVectorizer(encoding='gb18030',min_df=0.015)
    tfidftransformer = TfidfTransformer()
    countvector = countvectorizer.fit_transform(corpus)
    print(countvector.shape)
    tfidf = tfidftransformer.fit_transform(countvector)
    print(tfidf.shape)

    # 保存到文件，方便下次调用
    with open(path+'tfidf.pkl','wb') as file:
        pickle.dump(tfidf, file)
else:
    # 调用上次处理的结果
    with open(path+'tfidf.pkl','rb') as file:
        tfidf = pickle.load(file)

#print(type(tfidf))
# 将csr_matrix 转换为 numpy.ndarray类型, 同时将原来float64类型转换为float32类型
tfidf = tfidf.toarray().astype(np.float32)
# embedding的维度
d = tfidf.shape[1]
print(d)
print(tfidf.shape)
print(type(tfidf))
#print(tfidf[1])
print(type(tfidf[1][1]))

         id author     source content  \
100   89517    NaN  中国证券报?中证网     NaN   
103   89514    NaN  中国证券报?中证网     NaN   
997   88620    NaN        央广网     NaN   
1273  88344    NaN        央广网     NaN   
1282  88335    NaN        央广网     NaN   

                                                feature  \
100   {"type":"公司","site":"中证网","commentNum":"0","jo...   
103   {"type":"公司","site":"中证网","commentNum":"0","jo...   
997   {"type":"时事要闻","site":"参考消息","commentNum":"0",...   
1273  {"type":"IT业界","site":"参考消息","commentNum":"0",...   
1282  {"type":"IT业界","site":"参考消息","commentNum":"0",...   

                                 title  \
100       天和防务股东未来6个月内计划减持不超过480万股公司股份   
103                    晶盛机电调整限制性股票回购价格   
997              [主播不在家]第二季：主播陈亮体验垃圾清运   
1273                LKK洛可可：想象力经济时代或已到来   
1282  CES2017：京东发布两款叮咚智能音箱新品 开放Alpha平台   

                                                    url  
100   http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...  
103   http://www.cs.com.cn/ss

In [4]:
# 精确索引
import faiss
index = faiss.IndexFlatL2(d)  # 构建 IndexFlatL2
print(index.is_trained)  # False时需要train
index.add(tfidf)  #添加数据
print(index.ntotal)  #index中向量的个数

#精确索引无需训练便可直接查询
k = 10  # 返回结果个数
cpindex = 3352
query_self = tfidf[cpindex:cpindex+1]  # 查询本身
dis, ind = index.search(query_self, k)
print(dis.shape) # 打印张量 (5, 10)
print(ind.shape) # 打印张量 (5, 10)
print(dis)  # 升序返回每个查询向量的距离
print(ind)  # 升序返回每个查询向量

print('怀疑抄袭:\n', news.iloc[cpindex].content)
# 找一篇相似的原文
similar2 = ind[0][1]
print(similar2)
print('相似原文:\n', news.iloc[similar2].content)


True
87054
(1, 10)
(1, 10)
[[0.         0.06301732 0.07985516 0.10713604 0.11433167 0.24756216
  0.24929689 0.26127344 0.26870337 0.27568278]]
[[ 3352  3134  3125 63511 29441  3218   980 29615  4062 29888]]
怀疑抄袭:
 　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。
　　三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加
　　5月份，70个城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个。二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个。

3134
相似原文:
 　　国家统计局19日发布数据，5月份，15个一线